In [ ]:
using ESDL
using ESDLPlots
gr()

In [ ]:
ds=Cube()
d=getCubeData(ds,variable=["Biosphere"],region="Poland")
d2=d

In [ ]:
include("loadLC.jl")

In [ ]:
@loadOrGenerate v1=>"variance_pca" r1=>"rotation_pca" proj1=>"Projected" begin
  p1=cubePCA(d,noutdims=3)
  v1=explained_variance(p1)
  r1=rotation_matrix(p1)
  proj1=transformPCA(p1,d)  
end

In [ ]:
plotXY(v1,xaxis="pc")

In [ ]:
plotlyjs()
plotXY(r1,xaxis="pc",group="variable")

In [ ]:
proj1

In [ ]:
using OnlineStats
index = reduceCube(mean,proj1,(LonAxis,LatAxis),skipmissing=true)

In [ ]:
value=Reactive.value
plotTS(index,group="PC")

In [ ]:
@loadOrGenerate vlc=>"variance_by_landCover" rlc=>"rotation_by_landcover" begin
  plandclass=cubePCA(d,noutdims=3,by=[lc])
  vlc=explained_variance(plandclass)
  rlc=rotation_matrix(plandclass)
end

In [ ]:
plotXY(vlc,xaxis="pc",group="landcover")

In [ ]:
plotXY(rlc,xaxis="pc",group="variable")

Masks don't have to be spatial

In [ ]:
tax = d.timeAxis
function getSeason(x::Dates.TimeType)
  m = Dates.month(x)
  in(m,(12,1,2)) ? 1 : in(m,(3,4,5)) ? 2 : in(m,(6,7,8)) ? 3 : 4
end
seasonmask=CubeMem(CubeAxis[tax],getSeason.(tax.values),zeros(UInt8,length(tax)))
seasonmask.properties["labels"]=Dict(1=>"Spring",2=>"Summer",3=>"Autumn",4=>"Winter")
seasonmask.properties["name"]="Season"
plotTS(seasonmask)

In [ ]:
@loadOrGenerate vseason=>"variance_by_season" rseason=>"rotation_by_season" projseason=>"Projected_by_season" begin
  pseason=cubePCA(d,noutdims=3,by=[seasonmask])
  vseason=explained_variance(pseason)
  rseason=rotation_matrix(pseason)
  projseason=transformPCA(pseason,d)
end

In [ ]:
plotXY(vseason,xaxis="PC",group="season")

In [ ]:
plotXY(rseason,xaxis="PC",group="variable")

In [ ]:
@loadOrGenerate vtime=>"timePCA_variance" rtime=>"timePCA_rotation" projtime=>"timePCA_Projected" begin
  ptime = cubePCA(d,noutdims=5,by=[VariableAxis],MDAxis=TimeAxis)
  vtime  = explained_variance(ptime)
  rtime  = rotation_matrix(ptime)
  projtime = transformPCA(ptime,d)
end

In [ ]:
plotXY(vtime,xaxis="pc",group="variable")

In [ ]:
plotTS(rtime,pc=1,group="var")